<a href="https://colab.research.google.com/github/aerjayc/coe197z-hw1/blob/master/cifar_10_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape((-1,32*32*3))
x_test = x_test.reshape((-1,32*32*3))

# normalization, vectorization
from keras.utils import to_categorical

x_train = x_train.astype('float32') / 255
y_train = to_categorical(y_train)

x_test = x_test.astype('float32') / 255
y_test = to_categorical(y_test)

In [0]:
from keras import models
from keras import layers
from keras import regularizers

# input parameters
input_dim = 32*32*3
num_labels = 10

# network parameters
batch_size = 512
hidden_units = 1024
epochs = 41

dropout_rate = 0.3

model = models.Sequential()
model.add(layers.Dense(hidden_units,
                       activation='relu', 
                       input_dim=input_dim))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(hidden_units,
                       activation='relu'))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(hidden_units,
                       activation='relu'))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(num_labels, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 1024)              3146752   
_________________________________________________________________
dropout_34 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_82 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_35 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_83 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_36 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_84 (Dense)             (None, 10)              

In [0]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/41
50000/50000 [==============================] - 3s 58us/step - loss: 5.2594 - acc: 0.1488
Epoch 2/41
50000/50000 [==============================] - 2s 36us/step - loss: 2.0634 - acc: 0.2452
Epoch 3/41
50000/50000 [==============================] - 2s 37us/step - loss: 1.9767 - acc: 0.2819
Epoch 4/41
50000/50000 [==============================] - 2s 36us/step - loss: 1.9138 - acc: 0.3089
Epoch 5/41
50000/50000 [==============================] - 2s 36us/step - loss: 1.8660 - acc: 0.3276
Epoch 6/41
50000/50000 [==============================] - 2s 36us/step - loss: 1.8313 - acc: 0.3455
Epoch 7/41
50000/50000 [==============================] - 2s 37us/step - loss: 1.8031 - acc: 0.3519
Epoch 8/41
50000/50000 [==============================] - 2s 36us/step - loss: 1.7688 - acc: 0.3682
Epoch 9/41
50000/50000 [==============================] - 2s 37us/step - loss: 1.7482 - acc: 0.3753
Epoch 10/41
50000/50000 [==============================] - 2s 36us/step - loss: 1.7352 - acc: 0.3789

In [0]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 120us/step


[1.488067834663391, 0.491]